In [4]:
import pymongo
import pprint
import urllib.parse


In [17]:
# Replace XXXX with your connection URI from the Atlas UI
username = urllib.parse.quote_plus("Jacobninan")
password = urllib.parse.quote_plus("Syndicate@1234")

free_tier_client = pymongo.MongoClient("mongodb+srv://%s:%s@cluster0-tpvbw.gcp.mongodb.net/test?retryWrites=true&w=majority" % (username,password))


In [18]:
pprint.pprint(people.index_information())

{'_id_': {'key': [('_id', 1)], 'ns': 'cleansing.people-raw', 'v': 2}}


In [7]:
# We're using the people-raw dataset from the Cleansing Data with Updates assessment
people = free_tier_client.cleansing["people-raw"]

In [8]:
# This is a helper function to reduce the output of explain to a few key metrics
def distilled_explain(explain_output):
    return {
        'executionTimeMillis': explain_output['executionStats']['executionTimeMillis'],
        'totalDocsExamined'  : explain_output['executionStats']['totalDocsExamined'],
        'nReturned'          : explain_output['executionStats']['nReturned']
    }

In [9]:
query_1_stats = people.find({
  "address.state": "Nebraska",
  "last_name": "Miller",
}).explain()

query_2_stats = people.find({
  "first_name": "Harry",
  "last_name": "Reed"
}).explain()

In [10]:
# This is to provide a baseline for how long it takes to execute these queries
print(distilled_explain(query_1_stats))
print(distilled_explain(query_2_stats))

{'executionTimeMillis': 38, 'totalDocsExamined': 50474, 'nReturned': 6}
{'executionTimeMillis': 27, 'totalDocsExamined': 50474, 'nReturned': 1}


In [32]:
# Replace "YYYY" with the best index to increase the performance of the two queries above
#people.create_index([("address",pymongo.ASCENDING)])
people.create_index([("last_name",pymongo.ASCENDING)])


'last_name_1'

In [33]:
query_1_stats = people.find({
  "address.state": "Nebraska",
  "last_name": "Miller",
}).explain()

query_2_stats = people.find({
  "first_name": "Harry",
  "last_name": "Reed"
}).explain()

In [34]:
# If everything went well, both queries should now have *much* lower execution times and documents examined
print(distilled_explain(query_1_stats))
print(distilled_explain(query_2_stats))

{'executionTimeMillis': 4, 'totalDocsExamined': 533, 'nReturned': 6}
{'executionTimeMillis': 0, 'totalDocsExamined': 17, 'nReturned': 1}
